In [7]:
import numpy as np
from PIL import Image
import os
import random
from layeris.layer_image import LayerImage

In [8]:
class Mosaic:
    def __init__(self, path, slice_size):
        self.path = path
        # Every x (slice_size) pixels of the image should be another image
        self.slice_size = slice_size
        self.ref = Image.open(path).convert('RGB')
        # Resize the image such that every sub-image is of size 64x64px
        self.ref = self.ref.resize((int(self.ref.size[0] * 64 / self.slice_size), int(self.ref.size[1] * 64 / self.slice_size)))
        self.array = np.array(self.ref)
        self.x = self.ref.size[0]
        self.y = self.ref.size[1]
        self.content = []
            
    def load_content(self, path):
        for fn in os.listdir(path):
            image = Image.open(path + '/' + fn).resize((64, 64)).convert('RGB')
            self.content.append(image)
    
    def build(self):
        if not self.content:
            raise ValueError('No content loaded')
        # Iterate over blocks of the image
        for i in range (int(np.ceil(self.y/64))):
            for j in range(int(np.ceil(self.x/64))):
                [start_x, end_x] = [i*64, (i+1)*64]
                [start_y, end_y] = [j*64, (j+1)*64]
                arr_slice = self.array[start_x:end_x, start_y:end_y]
                # Obtain the average color of the block
                average = np.average(arr_slice.reshape((arr_slice.shape[0]*arr_slice.shape[1], 3)), axis=0).astype(np.uint8)
                self.array[start_x:end_x, start_y:end_y] = average
                # Generate tinted image to match the average color
                recolored_image = LayerImage.from_array(np.array(random.choice(self.content))/255).grayscale().vivid_light(average/255, 0.3).multiply(average/255, 0.7).brightness(0.4)
                # Paste the tinted image into the block
                self.array[start_x:end_x, start_y:end_y] = (recolored_image.get_image_as_array()*255)[:arr_slice.shape[0], :arr_slice.shape[1]]
        # Update PIL image
        self.ref = Image.fromarray(self.array)
    
  

In [11]:
image = Mosaic('source/ref/example.jpg', 12)
image.load_content('source/content')
image.build()

In [12]:
image.ref.show()

In [13]:
image.ref.save('out/result.jpg')